In [1]:
from langchain_community.document_loaders import PyPDFLoader

In [2]:
loader = PyPDFLoader ("PDF/vscode.pdf")

pages = []
async for page in loader.alazy_load():
    pages.append (page)

Multiple definitions in dictionary at byte 0x299e for key /ExtGState
Multiple definitions in dictionary at byte 0x2b37 for key /ExtGState
Multiple definitions in dictionary at byte 0x2f72 for key /ExtGState
Multiple definitions in dictionary at byte 0x317b for key /ExtGState
Multiple definitions in dictionary at byte 0x287 for key /ExtGState
Multiple definitions in dictionary at byte 0x40d for key /ExtGState
Multiple definitions in dictionary at byte 0x594 for key /ExtGState


In [19]:
pages[0].metadata['page_label']

'1'

In [11]:
import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb
from more_itertools import chunked
from tqdm import tqdm

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
client = chromadb.CloudClient(
  api_key = os.getenv ("CHROMA_API_KEY"),
  tenant = os.getenv ("CHROMA_TENANT"),
  database = 'ChromaDB'
)

In [9]:
embedding_model = HuggingFaceEmbeddings(
        model_name="BAAI/bge-large-en-v1.5",
        model_kwargs={'device': "cpu"}
    )

In [10]:
vectorStore = Chroma (
    collection_name="test_embedding",
    embedding_function=embedding_model,
    client=client
)

In [12]:
spliter = RecursiveCharacterTextSplitter (chunk_size=512, chunk_overlap=64)

In [16]:
async def embedPDF (path: str):
    
    for pdf in tqdm (os.listdir(path), desc="Processing PDFs"):
        if pdf.endswith (".pdf"):
            pdfPath = os.path.join(path, pdf)
            loader = PyPDFLoader (pdfPath)
            pages:list

            async for page in loader.alazy_load ():
                doc = spliter.split_documents ([page])
                success = False
                while not success:
                    try:
                        vectorStore.add_documents(doc)
                        success = True
                    except:
                        print ("Retrying page")
            print (f"Done embedding {pdf}")


In [17]:
path = "PDF"
await embedPDF (path=path)

Processing PDFs:   0%|          | 0/1 [00:00<?, ?it/s]Multiple definitions in dictionary at byte 0x299e for key /ExtGState
Multiple definitions in dictionary at byte 0x2b37 for key /ExtGState
Multiple definitions in dictionary at byte 0x2f72 for key /ExtGState
Multiple definitions in dictionary at byte 0x317b for key /ExtGState
Multiple definitions in dictionary at byte 0x287 for key /ExtGState
Multiple definitions in dictionary at byte 0x40d for key /ExtGState
Multiple definitions in dictionary at byte 0x594 for key /ExtGState
Processing PDFs: 100%|██████████| 1/1 [01:38<00:00, 98.86s/it]

Done embedding vscode.pdf
